In [29]:
!pip install neo4j langchain wikipedia openai tiktoken

## Neo4j Environment setup
You need to setup a Neo4j 5.11 or greater. 

In [39]:
from langchain.graphs import Neo4jGraph
NEO4J_URI="neo4j+s://"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=""
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)


## Setting up the Vector Index

In [ ]:
graph.query("""
CALL db.index.vector.createNodeIndex(
  'wikipedia', // index name
  'Chunk',     // node label
  'embedding', // node property
   1536,       // vector size
   'cosine'    // similarity metric
)
""")

## Populating the Vector index

Neo4j is schema-less by design, which means it doesn't enforce any restrictions what goes into a node property. For example, the embedding property of the Chunk node could store integers, list of integers or even strings. Let's try this out.

In [5]:

graph.query("""
WITH [1, [1,2,3], ["2","5"], [x in range(0, 1535) | toFloat(x)]] AS exampleValues
UNWIND range(0, size(exampleValues) - 1) as index
CREATE (:Chunk {embedding: exampleValues[index], index: index})
""")

[]

This query creates a Chunknode for each element in the list and uses the element as the embeddingproperty value. For example, the first Chunk node will have the embedding property value 1, the second node [1,2,3], and so on. Neo4j doesn't enforce any rules on what you can store under node properties. However, the vector index has clear instructions about the type of values and their embedding dimension it should index.
We can test which values were indexed by performing a vector index search.

In [6]:

graph.query("""
CALL db.index.vector.queryNodes(
  'wikipedia', // index name
  3, // topK neighbors to return
  [x in range(0,1535) | toFloat(x) / 2] //input vector
)
YIELD node, score
RETURN node.index AS index, score
""")

[{'index': 3, 'score': 1.0}]

If you run this query, you will get only a single node returned, even though you requested the top 3 neighbors to be returned. Why is that so? The vector index only indexes property values, where the value is a list of floats with the specified size. In this example, only one embeddingproperty value had the list of floats type with the selected length 1536.
A node is indexed by the vector index if all the following are true:
* The node contains the configured label.
* The node contains the configured property key.
* The respective property value is of type LIST<FLOAT>.
* The size() of the respective value is the same as the configured dimensionality.
* The value is a valid vector for the configured similarity function.

In [7]:

graph.query("""
MATCH (n) DETACH DELETE n
""")

[]

## Integrating the vector index into the LangChain ecosystem

The task will consist of the following steps:

* Retrieve a Wikipedia article
* Chunk the text
* Store the text along with its vector representation in Neo4j
* Implement a custom LangChain class to support RAG applications

In [8]:

import wikipedia

bg3 = wikipedia.page(pageid=60979422)

In [9]:

print(bg3.content)

Baldur's Gate 3 is a role-playing video game developed and published by Larian Studios. It is the third main game in the Baldur's Gate series, which is based on the Dungeons & Dragons tabletop role-playing system. A partial version of the game was released in early access format for macOS, Windows, and the Stadia streaming service, on 6 October 2020. The game remained in early access until its full release on Windows on 3 August 2023. The PlayStation 5 version was released on 6 September 2023, macOS version is scheduled for release on 21 September 2023, and the Xbox Series X/S version is planned for 2023.
Baldur's Gate 3 was acclaimed by critics, who praised the gameplay, narrative, amount of content, and player choice.


== Gameplay ==
Baldur's Gate 3 is a role-playing video game with a single-player and cooperative multiplayer element. Players can create one or more characters and form a party along with a number of pre-generated characters to explore the game's story. Optionally, pl

We need to chunk and embed the text. We will split the text by section using the double newline delimiter and then use OpenAI's embedding model to represent each section with an appropriate vector.

In [14]:

import os
from langchain.embeddings import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = ""

embeddings = OpenAIEmbeddings()

chunks = [{'text':el, 'embedding': embeddings.embed_query(el)} for
                  el in bg3.content.split("\n\n") if len(el) > 50]

## Import the text chunks into Neo4j
db.create.setVectorProperty procedure is used to store the vectors to Neo4j. This procedure is used to verify that the property value is indeed a list of floats. Additionally, it has the added benefit of reducing the storage space of vector property by approximately 50%. Therefore, it is recommended always to use this procedure to store vectors to Neo4j.

In [15]:

graph.query("""
UNWIND $data AS row
CREATE (c:Chunk {text: row.text})
WITH c, row
CALL db.create.setVectorProperty(c, 'embedding', row.embedding)
YIELD node
RETURN distinct 'done'
""", {'data': chunks})

[{"'done'": 'done'}]

## Neo4jVectorChain
When you can call the Neo4jVectorChain, the following steps are executed:

* Embed the question using the relevant embedding model
* Use the text embedding value to retrieve most similar content from the vector index
* Use the provided context from similar content to generate the answer

In [26]:
from langchain.chains.base import Chain
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering.stuff_prompt import CHAT_PROMPT
from langchain.callbacks.manager import CallbackManagerForChainRun

from typing import Any, Dict, List
from pydantic import Field

vector_search = """
WITH 
k, e) yield node, score
RETURN node.text AS result
ORDER BY score DESC
LIMIT 3
"""

class Neo4jVectorChain(Chain):
    """Chain for question-answering against a neo4j vector index."""

    graph: Neo4jGraph = Field(exclude=True)
    input_key: str = "query"  #: :meta private:
    output_key: str = "result"  #: :meta private:
    embeddings: OpenAIEmbeddings = OpenAIEmbeddings()
    qa_chain: LLMChain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=CHAT_PROMPT)

    @property
    def input_keys(self) -> List[str]:
        """Return the input keys.
        :meta private:
        """
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        """Return the output keys.
        :meta private:
        """
        _output_keys = [self.output_key]
        return _output_keys

    def _call(self, inputs: Dict[str, str], run_manager, k=3) -> Dict[str, Any]:
        """Embed a question and do vector search."""
        question = inputs[self.input_key]
        embedding = self.embeddings.embed_query(question)
        run_manager.on_text(
            "Vector search embeddings:", end="\n", verbose=self.verbose
        )
        run_manager.on_text(
            embedding[:5], color="green", end="\n", verbose=self.verbose
        )

        context = self.graph.query(
            vector_search, {'embedding': embedding, 'k': 3})
        context = [el['result'] for el in context]
        run_manager.on_text(
            "Retrieved context:", end="\n", verbose=self.verbose
        )
        run_manager.on_text(
            context, color="green", end="\n", verbose=self.verbose
        )

        result = self.qa_chain(
            {"question": question, "context": context},
        )
        final_result = result[self.qa_chain.output_key]
        return {self.output_key: final_result}

In [27]:
vector_qa = Neo4jVectorChain(graph=graph, embeddings=embeddings, verbose=True)

In [ ]:
vector_qa.run("What is the gameplay of Baldur s Gate 3 like?")

## Langchain examples

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

## Seeding the database

In [34]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

## Refresh graph schema information

In [35]:
graph.refresh_schema()

In [36]:
print(graph.get_schema)


        Node properties are the following:
        [{'labels': 'Chunk', 'properties': [{'property': 'embedding', 'type': 'LIST'}, {'property': 'text', 'type': 'STRING'}]}, {'labels': 'Movie', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'Actor', 'properties': [{'property': 'name', 'type': 'STRING'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        ['(:Actor)-[:ACTED_IN]->(:Movie)']
        


## Querying the graph

In [37]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [40]:
chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name
Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


'Tom Cruise, Val Kilmer, Anthony Edwards, and Meg Ryan played in Top Gun.'

## Return intermediate results

In [41]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_intermediate_steps=True
)

In [42]:
result = chain("Who played in Top Gun?")
print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name
Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.
Intermediate steps: [{'query': "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})\nRETURN a.name"}, {'context': [{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]}]
Final answer: Tom Cruise, Val Kilmer, Anthony Edwards, and Meg Ryan played in Top Gun.


## Add examples in the Cypher generation prompt

In [43]:
from langchain.prompts.prompt import PromptTemplate


CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, cypher_prompt=CYPHER_GENERATION_PROMPT
)

In [44]:
chain.run("How many people played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {name:"Top Gun"})<-[:ACTED_IN]-(:Actor)
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 4}]

> Finished chain.


'Four people played in Top Gun.'